# Basic setup
---
dependency codes for flask.

In [1]:
# !pip install flask
# !pip install flask_restful
# !pip install requests-oauthlib==1.1.0
# !pip install oauthlib==2.1.0
# !pip install xlrd
# !pip install xlwt

In [2]:
# flask server setup import
from flask import Flask, render_template, request, redirect, url_for, Response
from flask_restful import Resource, Api, reqparse
import sys

# fitbit api linker setup
from fitbit.api import Fitbit
from oauthlib.oauth2.rfc6749.errors import MismatchingStateError, MissingTokenError, InvalidClientError
from queue import Queue

# data fetching pandas / np setup
import pandas as pd 
import numpy as np
import json
import datetime
from datetime import timedelta  
import time
import threading

## Flask Web Server setting
---
Constructing Fitbit Application Client. 

1. CLIENT_ID: client id of fitbit app (see manage app in dev.fitbit.com)
2. CLIENT_SECRET: client secret of fitbit app
3. REDIRECT_URI: redirection url set on fitbit app 

In [3]:
TOKEN_F_NAME = r'data/auth.json'

def get_token():
    _token_f = open(TOKEN_F_NAME)
    token = json.loads(_token_f.read())
    _token_f.close()
    return token

def get_token_list(token_dict):
    token_list = []
    for key, val in token_dict.items():
        val['userID'] = key
        token_list.append(val)
    return token_list
    
def write_token(token_data):
    _token_f = open(TOKEN_F_NAME, 'w')
    json.dump(token_data, _token_f)
    _token_f.close()

In [4]:
import os

# static variables.
CLIENT_ID = os.environ['ID']
CLIENT_SECRET = os.environ['SECRET']
REDIRECT_URI = "https://fitbit.run-us-west1.goorm.io/auth"

class OAuth2Server:
    def __init__(self, client_id, client_secret, redirect_uri):
        self.success_html = """
        <hr> You are now authorized to access the Fitbit API!</h3>
        <br/><h3> You can close this window now </h3>"""
        
        self.failure_html = """
        <h1> ERROR: </h1><br/><h3> You can close the window now"""
        self.fitbit = Fitbit(
            client_id,
            client_secret,
            redirect_uri=redirect_uri,
            timeout=10,
        )
        
        print(redirect_uri)
    
    def auth_url(self):
        print(self.fitbit.client.authorize_token_url())
        return self.fitbit.client.authorize_token_url()
    
app = Flask(__name__)
api = Api(app)
server = OAuth2Server(CLIENT_ID, CLIENT_SECRET, REDIRECT_URI)

https://fitbit.run-us-west1.goorm.io/auth


In [5]:
# APIs
@app.route('/', methods=['GET'])
def index():
    token_dict = get_token()
#     return url
    return render_template('index.html', url='add_user', users=get_token_list(token_dict))

@app.route('/add_user', methods=['GET'])
def add():
    url, _ = server.auth_url()
    print(url)
    
#     return url
    return redirect(url, code=301)

@app.route('/auth', methods=['GET'])
def auth():
    code = request.args.get('code', type=
                           str)
    state = request.args.get('state', type=str)
    error = request.args.get('error', type=str)
    
#     return code
    if code:
        try:
            token_dict = get_token()
            token = server.fitbit.client.fetch_access_token(code)
            ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
            REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
            
            ##Build Auth Clients
            auth2_client = Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')

            #NEed this seeparately because version 1 doesn't give sleep breakdown
            auth2_client_new = Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')
            auth2_client_new.API_VERSION = 1.2
            profile = auth2_client.user_profile_get()
            print(profile)
            user_ID = profile['user']['encodedId']
            user_name = profile['user']['fullName']
            token = {
                'userName': user_name,
                'token': ACCESS_TOKEN,
                'refreshToken': REFRESH_TOKEN
            }
            
            token_dict[user_ID] = token
            print(token_dict)
            write_token(token_dict)
            return render_template('auth.html')
        
        except MissingTokenError:
            error = server._fmt_failure(
                'Missing access token parameter.</br>Please check that '
                'you are using the correct client_secret')
        except MismatchingStateError:
            error = server._fmt_failure('CSRF Warning! Mismatching state')
#         except InvalidClientError:
#             return code

@app.route('/fetch_test', methods=['GET'])
def fetch_test():
    token_dict = get_token()
    #temp for implementation
    ##Build Auth Clients
    ACCESS_TOKEN = list(token_dict.values())[1]['token']
    REFRESH_TOKEN = list(token_dict.values())[1]['refreshToken']
    auth2_client = Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')

    #NEed this seeparately because version 1 doesn't give sleep breakdown
    auth2_client_new = Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')
    auth2_client_new.API_VERSION = 1.2
    return auth2_client.user_profile_get()

In [ ]:
app.run(host="0.0.0.0", threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [26/Dec/2019 07:44:06] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [26/Dec/2019 07:48:37] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [26/Dec/2019 07:48:39] "GET / HTTP/1.1" 200 -
